In [1]:
# !pip install xlrd
# !pip install pandas
import numpy as np
import pandas as pd
import random
import re

from pandas import Series, DataFrame
from collections import Counter

In [2]:
df_train = pd.read_excel('log_data_evil.xlsx', index_col=0)
df_train.sort_index(ascending=True, inplace=True)

df_train.drop(['http_user_agent'], inplace=True, axis=1)

In [3]:
df_train.request_url.replace({r'^/$': '<BASE_URL>'}, regex=True, inplace=True)
df_train.request_url.replace({r' \%[\%0-9A-Za-z]*': ' <PERCENT_URL> '},regex=True, inplace=True)
df_train.request_url.replace({r'\w+\d+\w+': ''},regex=True, inplace=True)
df_train.request_url.replace({r'\d{2,}': ''},regex=True, inplace=True)
df_train.request_url.replace({r'\[\d*\]': ' '},regex=True, inplace=True)
df_train.request_url.replace({'/': ' ', ':': ' ', '\.': ' ', '\?': ' ', '=': ' ', '\|': ' ', '&': ' '},regex=True, inplace=True)

In [4]:
list_aux = df_train['request_url'].tolist()

for i in range (len(list_aux)):
    list_aux[i] = ' '.join(list_aux[i].split()).lower()

In [5]:
as_one = ' '.join(list_aux)

words = as_one.split()

counts = Counter(words)
vocab = sorted(counts, key=counts.get)

word_index = {word: (i+8) for i, word in enumerate(vocab, 1)}
word_index['<PAD>'] = 0
word_index["<URL_START>"] = 1
word_index['<GET>'] = 2
word_index['<POST>'] = 3
word_index['<TEMPORARY_REDIRECT>'] = 4
word_index['<BAD_REQUEST>'] = 5
word_index['<NOT_FOUND>'] = 6
word_index['<OK>'] = 7
word_index['<MOVED_PERMANTLY>'] = 8


reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode(ls):
    return ' '.join([reverse_word_index.get(i, '?') for i in ls])

def encode(text):
    text_ls = []
    for i in text.split():
        try:
            text_ls.append(word_index[i])
        except KeyError:
            print('Unhandled word \'' + i + '\'')
    return text_ls

def decode_single(text):
    return reverse_word_index.get(text)

def encode_single(text):
    return word_index.get(text)

def insert_start(ls):
    ls.insert(0, 1)
    return ls

In [30]:
import pickle

with open('abd_variables.pkl', 'wb') as f:
    pickle.dump([word_index, reverse_word_index], f)

In [6]:
df_train['request_url'] = df_train['request_url'].str.lower()
df_train['request_url'] = df_train['request_url'].apply(encode)
df_train['request_url'] = df_train['request_url'].apply(insert_start)

In [7]:
df_train['request_method'].replace({'GET': 2, 'POST': 3}, inplace=True)

In [8]:
df_train['status'].replace({307: '<TEMPORARY_REDIRECT>', 400: '<BAD_REQUEST>', 404: '<NOT_FOUND>', 200: '<OK>', 301: '<MOVED_PERMANTLY>'}, inplace=True)
df_train['status'] = df_train['status'].apply(encode_single)

In [9]:
total_labels = df_train['is_evil'].values
df_train.drop('is_evil', axis=1, inplace=True)

In [10]:
max_lenght = 0
for i in df_train.values:
    if len(i[2]) > max_lenght:
        max_lenght = len(i[2])

print('the largest url have', max_lenght, 'words')

the largest url have 11 words


In [11]:
# from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

1.14.0
['/device:CPU:0', '/device:XLA_CPU:0']


In [12]:
aux_request_data = keras.preprocessing.sequence.pad_sequences(df_train['request_url'].values,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=max_lenght)
df_train['request_url'] = Series(aux_request_data.tolist())

In [13]:
for i in range(max_lenght):
    df_train['request_url' + str(i)] = 0
    
for i in df_train.itertuples():
    for j in range(max_lenght):
        df_train.at[i[0], 'request_url' + str(j)] = i[3][j]

df_train.drop(['request_url'], axis=1, inplace=True)

total_data = df_train.values

In [14]:
print(total_data.shape, total_labels.shape)
print(len(total_data), len(total_labels))

vocab_size = len(word_index.keys())

vocab_size

(7716, 13) (7716,)
7716 7716


742

In [15]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

W0711 09:05:33.437913 140206357854016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0711 09:05:33.483238 140206357854016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          11872     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 12,161
Trainable params: 12,161
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

W0711 09:05:33.539999 140206357854016 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
train_data = total_data[:3858]
train_labels = total_labels[:3858]

In [18]:
test_data = total_data[3858:]
test_labels = total_labels[3858:]

In [19]:
print(len(train_data), len(train_labels))
print(len(test_data), len(test_labels))

3858 3858
3858 3858


In [20]:
x_val = train_data[:1500]
partial_x_train = train_data[1500:]

y_val = train_labels[:1500]
partial_y_train = train_labels[1500:]

In [21]:
print(len(partial_x_train), len(partial_y_train))
print(len(x_val), len(y_val))

2358 2358
1500 1500


In [22]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=200,
                    batch_size=22,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 2358 samples, validate on 1500 samples
Epoch 1/200
2358/2358 [==============================] - 0s 93us/sample - loss: 0.5604 - acc: 0.8813 - val_loss: 0.4506 - val_acc: 0.8373
Epoch 2/200
2358/2358 [==============================] - 0s 49us/sample - loss: 0.3079 - acc: 0.9008 - val_loss: 0.3683 - val_acc: 0.8373
Epoch 3/200
2358/2358 [==============================] - 0s 53us/sample - loss: 0.2385 - acc: 0.9008 - val_loss: 0.2923 - val_acc: 0.8373
Epoch 4/200
2358/2358 [==============================] - 0s 52us/sample - loss: 0.1626 - acc: 0.9160 - val_loss: 0.1615 - val_acc: 0.9167
Epoch 5/200
2358/2358 [==============================] - 0s 53us/sample - loss: 0.0737 - acc: 0.9869 - val_loss: 0.0636 - val_acc: 0.9980
Epoch 6/200
2358/2358 [==============================] - 0s 53us/sample - loss: 0.0286 - acc: 1.0000 - val_loss: 0.0328 - val_acc: 0.9987
Epoch 7/200
2358/2358 [==============================] - 0s 51us/sample - loss: 0.0144 - acc: 1.0000 - val_loss: 0.0219 - va

2358/2358 [==============================] - 0s 48us/sample - loss: 2.2060e-05 - acc: 1.0000 - val_loss: 0.0058 - val_acc: 0.9987
Epoch 59/200
2358/2358 [==============================] - 0s 53us/sample - loss: 2.0578e-05 - acc: 1.0000 - val_loss: 0.0058 - val_acc: 0.9987
Epoch 60/200
2358/2358 [==============================] - 0s 50us/sample - loss: 1.9180e-05 - acc: 1.0000 - val_loss: 0.0058 - val_acc: 0.9987
Epoch 61/200
2358/2358 [==============================] - 0s 52us/sample - loss: 1.7914e-05 - acc: 1.0000 - val_loss: 0.0058 - val_acc: 0.9987
Epoch 62/200
2358/2358 [==============================] - 0s 50us/sample - loss: 1.6755e-05 - acc: 1.0000 - val_loss: 0.0058 - val_acc: 0.9987
Epoch 63/200
2358/2358 [==============================] - 0s 49us/sample - loss: 1.5686e-05 - acc: 1.0000 - val_loss: 0.0058 - val_acc: 0.9987
Epoch 64/200
2358/2358 [==============================] - 0s 51us/sample - loss: 1.4581e-05 - acc: 1.0000 - val_loss: 0.0058 - val_acc: 0.9987
Epoch 65/200

2358/2358 [==============================] - 0s 50us/sample - loss: 5.6879e-07 - acc: 1.0000 - val_loss: 0.0069 - val_acc: 0.9987
Epoch 116/200
2358/2358 [==============================] - 0s 50us/sample - loss: 5.3651e-07 - acc: 1.0000 - val_loss: 0.0069 - val_acc: 0.9987
Epoch 117/200
2358/2358 [==============================] - 0s 51us/sample - loss: 5.0659e-07 - acc: 1.0000 - val_loss: 0.0069 - val_acc: 0.9987
Epoch 118/200
2358/2358 [==============================] - 0s 49us/sample - loss: 4.7926e-07 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 0.9987
Epoch 119/200
2358/2358 [==============================] - 0s 48us/sample - loss: 4.5135e-07 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 0.9987
Epoch 120/200
2358/2358 [==============================] - 0s 49us/sample - loss: 4.2564e-07 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 0.9987
Epoch 121/200
2358/2358 [==============================] - 0s 53us/sample - loss: 4.0203e-07 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 0.9987
Epoch 

Epoch 172/200
2358/2358 [==============================] - 0s 59us/sample - loss: 2.4013e-08 - acc: 1.0000 - val_loss: 0.0084 - val_acc: 0.9987
Epoch 173/200
2358/2358 [==============================] - 0s 51us/sample - loss: 2.2866e-08 - acc: 1.0000 - val_loss: 0.0084 - val_acc: 0.9987
Epoch 174/200
2358/2358 [==============================] - 0s 48us/sample - loss: 2.1688e-08 - acc: 1.0000 - val_loss: 0.0085 - val_acc: 0.9987
Epoch 175/200
2358/2358 [==============================] - 0s 51us/sample - loss: 2.0601e-08 - acc: 1.0000 - val_loss: 0.0085 - val_acc: 0.9987
Epoch 176/200
2358/2358 [==============================] - 0s 52us/sample - loss: 1.9548e-08 - acc: 1.0000 - val_loss: 0.0086 - val_acc: 0.9987
Epoch 177/200
2358/2358 [==============================] - 0s 52us/sample - loss: 1.8576e-08 - acc: 1.0000 - val_loss: 0.0085 - val_acc: 0.9987
Epoch 178/200
2358/2358 [==============================] - 0s 55us/sample - loss: 1.7711e-08 - acc: 1.0000 - val_loss: 0.0086 - val_acc:

In [23]:
results = model.evaluate(test_data, test_labels)

3858/3858 [==============================] - 0s 12us/sample - loss: 7.0589e-05 - acc: 1.0000


In [24]:
model.save("abd_model.h5")
print("model saved to disk")

model saved to disk


In [25]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [26]:
def evaluate_log(log):
    test = StringIO(log)
    df_evaluate = pd.read_csv(test, sep=" ", header=None)
    try:
        df_evaluate[11], df_evaluate[12], df_evaluate[13] = df_evaluate[5].str.split(' ', 2).str
    except ValueError:
        return -1
    df_evaluate.drop([0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 13], axis=1, inplace=True)
    if df_evaluate.at[0,11] != 'GET' and df_evaluate.at[0,11] != 'POST':
        return -1
    
    df_evaluate[12].replace({r'^/$': '<BASE_URL>'}, regex=True, inplace=True)
    df_evaluate[12].replace({r' \%[\%0-9A-Za-z]*': ' <PERCENT_URL> '},regex=True, inplace=True)
    df_evaluate[12].replace({r'\w+\d+\w+': ''},regex=True, inplace=True)
    df_evaluate[12].replace({r'\d{2,}': ''},regex=True, inplace=True)
    df_evaluate[12].replace({r'\[\d*\]': ' '},regex=True, inplace=True)
    df_evaluate[12].replace({'/': ' ', ':': ' ', '\.': ' ', '\?': ' ', '=': ' ', '\|': ' ', '&': ' '},regex=True, inplace=True)
    df_evaluate[11].replace({'GET': 2, 'POST': 3}, inplace=True)
    
    df_evaluate[12] = df_evaluate[12].str.lower()
    df_evaluate[12] = df_evaluate[12].apply(encode)
    df_evaluate[12] = df_evaluate[12].apply(insert_start)
    
    df_evaluate[6].replace({307: '<TEMPORARY_REDIRECT>', 400: '<BAD_REQUEST>', 404: '<NOT_FOUND>', 200: '<OK>', 301: '<MOVED_PERMANTLY>'}, inplace=True)
    df_evaluate[6] = df_evaluate[6].apply(encode_single)
    
    df_evaluate.columns = ['status', 'method', 'r_url']
    
    if len(df_evaluate.at[0, 'r_url']) < 11:
        for i in range (len(df_evaluate.at[0,'r_url']), 11):
            df_evaluate.at[0,'r_url'].append(0)
            
    for i in range(11):
        df_evaluate.at[0, 'r_url' + str(i)] = df_evaluate.at[0, 'r_url'][i]

    df_evaluate.drop(['r_url'], axis=1, inplace=True)
    
    single_evaluate = df_evaluate.values[0]
    
    print('Handled URL:')
    print(decode(single_evaluate))
    
    single_evaluate = (np.expand_dims(single_evaluate, 0))

    return round(model.predict(single_evaluate)[0][0], 4)

In [27]:
log = '''
88.147.113.229 - - [06/Jul/2019:03:53:25 -0500] "GET /robots.ga HTTP/1.1" 400 173 "-" "Mozilla/5.0" "-"
'''

evaluate_log(log)

Unhandled word 'ga'
Handled URL:
<BAD_REQUEST> <GET> <URL_START> robots <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


1.0